# Experiment for the change in consensus when we add a sum_normalization step
for the PR
https://github.com/opentensor/subtensor/pull/415

In [1]:
import bittensor as bt
block = 2957500
sub = bt.subtensor('archive')
meta_0 = sub.metagraph(netuid = 0, lite = False, block=block)

2024-05-14 20:53:42.240 |       INFO       | Connected to archive network and wss://archive.chain.opentensor.ai:443/.


In [2]:
import torch
import torch.nn as nn

# === Live emission
def get_subnet_emission(block):
    emissions = []
    subnets = sub.get_all_subnets_info(block = block)
    for subnet in subnets:
        emissions.append(subnet.emission_value / bt.utils.RAOPERTAO)
    return torch.tensor(emissions)

# === Existing yuma consensus calculation
def Yuma1(W, S):
    rho = 10
    kappa = 0.5

    def normalize(W, S):
        S = S / S.sum()
        W = (W.T / (W.max(dim=1).values +  1e-6)).T
        W = (W * 65535).int()
        return W, S

    W, S = normalize(W, S)
    T = (S.view(-1 , 1) * ( W > 0 )).sum(dim =  0)
    C = 1 / (1 + torch.exp(-(rho * (T - kappa))))
    R = (S.view(-1 , 1) * W).sum(dim=0)
    I = C * R 
    
    return {
        'weight': W,
        'stake': S,
        'trust': T,
        'consensus': C,
        'rank': R,
        'incentive': I/I.sum()
    }

# === Existing yuma consensus calculation
def Yuma1_new(W, S):
    rho = 10
    kappa = 0.5

    def normalize_max(W):
        W = (W.T / (W.max(dim=1).values +  1e-6)).T
        W = (W * 65535).int()
        return W

    def normalize_sum(W):
        W = (W.T / (W.sum(dim=1))).T
        W = W.nan_to_num(0)
        W = (W * 65535).int()
        return W

    W = normalize_sum(normalize_max(W))
    S = S / S.sum()
    T = (S.view(-1 , 1) * ( W > 0 )).sum(dim =  0)
    C = 1 / (1 + torch.exp(-(rho * (T - kappa))))
    R = (S.view(-1 , 1) * W).sum(dim=0)
    I = C * R 
    
    return {
        'weight': W,
        'stake': S,
        'trust': T,
        'consensus': C,
        'rank': R,
        'incentive': I/I.sum()
    }

In [3]:
# === check that the existing calculation is right
live_emission = get_subnet_emission(block = meta_0.block.item())
stat = Yuma1(meta_0.W, meta_0.S)

criterion = nn.MSELoss()
loss = torch.sqrt(criterion(stat['incentive'], live_emission))

loss, loss / live_emission.mean()

(tensor(1.2506e-05), tensor(0.0004))

In [4]:
import pandas as pd

# === Check the difference in emission when yuma get updated
stat_new = Yuma1_new(meta_0.W, meta_0.S)
df = pd.DataFrame(torch.stack([stat['incentive'], stat_new['incentive']]).T, columns = ['old_emission', 'new_emission'])
df['change'] = (df.new_emission - df.old_emission)
df['change_in_percentage(%)'] = df.change / df.old_emission * 100
df.sort_values('change', ascending = False)

,old_emission,new_emission,change,change_in_percentage(%)
0,0.015307,0.046756,0.031449,205.454178
14,0.015126,0.034311,0.019185,126.838364
2,0.029204,0.040636,0.011433,39.147781
30,0.018908,0.027805,0.008897,47.055519
16,0.011110,0.016805,0.005696,51.266594
32,0.014706,0.017408,0.002702,18.373825
13,0.025632,0.027119,0.001486,5.798191
20,0.023969,0.024617,0.000648,2.704977
33,0.000047,0.000060,0.000012,26.376686
11,0.000075,0.000080,0.000006,7.450792
